In [1]:
import rasterio
from rasterio.mask import mask
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from src.ndvi import get_yard_data, calculate_ndvi

import warnings
warnings.filterwarnings("ignore")

In [2]:
def process_image(processed_yard_data, aeriel_rgb, aeriel_cir):
    """
    Calculate the NDVI and its mean for the aeriel images.
    Make sure that the bounds of the processed_yard_data are the same bounds as of the aeriel images.    

    Parameters
    ----------
    processed_yard_data : geopandas.GeoDataFrame
        The processed yard data with the geometry of the erf zones.
        Obtained from the get_yard_data function.
    aeriel_rgb : rasterio.io.DatasetReader
        The aeriel image in RGB format.
    aeriel_cir : rasterio.io.DatasetReader  
        The aeriel image in CIR format.
    
    Returns
    -------
    df : pandas.DataFrame
        A DataFrame with the yards NDVI and its mean for each erf zone.

    """
        # Create a list to store results
    plot_dict = {}

    # Iterate over each erf zone
    for _, row in processed_yard_data.iterrows():
        erf_id = row['id']
        plot_id = row['gml_id']
        zone_geometry = [row['geometry']]
        try:
            mask_image_cir, out_transform_cir = mask(aeriel_cir, zone_geometry, crop=True, filled=False)
            mask_image_rgb, _ = mask(aeriel_rgb, zone_geometry, crop=True, filled=False)
            # Store results (zone_id and clipped raster)
            plot_dict[plot_id] = {
                'erf_id': erf_id,
                "clipped_cir": mask_image_cir,
                "clipped_rgb": mask_image_rgb,
                "affine_transform": out_transform_cir
            }
        except ValueError:
            pass

    df = pd.DataFrame(plot_dict).T
    # apply the NDVI to df 
    df['ndvi'] = df.apply(lambda x: calculate_ndvi(x['clipped_cir']), axis=1)

    # Calculate the ndvi mean over unmasked pixels
    df['ndvi_mean'] = df['ndvi'].apply(lambda x: np.ma.mean(x))

    return df

In [ ]:
years = [2020, 2022, 2023]
results = []
for count, year in enumerate(years):
    # load the aerial images and get the geo bounds
    input_aerial_rgb = f'data/{year}_breda_rgb_small.tif'
    input_aerial_cir = f'data/{year}_breda_cir_small.tif'
    aeriel_rgb = rasterio.open(input_aerial_rgb)
    aeriel_cir = rasterio.open(input_aerial_cir)
    if count > 0:
        assert aeriel_cir.bounds == bounds, "The bounds of the images are not the same over the years"
    bounds = tuple(aeriel_cir.bounds)
    if count == 0:
        largest_overlap = get_yard_data(bounds)
    results.append(process_image(largest_overlap, aeriel_rgb, aeriel_cir))

In [ ]:
# make a ndvi histogram
for count, df in enumerate(results):
    plt.hist(df['ndvi_mean'], bins=40, alpha=0.5, label=years[count])
    plt.xlabel('NDVI mean per yard')
    plt.ylabel('Frequency')
plt.legend()

In [ ]:
# plot top 8 plots with lowest NDVI mean and directly below that plot the correspondig NDVI map
top5 = df.sort_values(by='ndvi_mean').head(8)
fig, ax = plt.subplots(2, 8, figsize=(20, 10))
for i, (_, row) in enumerate(top5.iterrows()):
    ax[0, i].imshow(row['clipped_cir'].transpose((1, 2, 0)))
    ax[0, i].set_title(f"NDVI mean: {row['ndvi_mean']:.2f}")
    ax[0, i].axis('off')
    ax[1, i].imshow(row['ndvi'], cmap='coolwarm', vmin=-0.5, vmax=0.5)
    ax[1, i].axis('off')
plt.show()

In [ ]:
# plot top 5 plots with highest NDVI mean and directly below that plot the correspondig NDVI map
top5 = df.nlargest(8, 'ndvi_mean')
fig, ax = plt.subplots(2, 8, figsize=(20, 10))
for i, (_, row) in enumerate(top5.iterrows()):
    ax[0, i].imshow(row['clipped_cir'].transpose((1, 2, 0)))
    ax[0, i].set_title(f"NDVI mean: {row['ndvi_mean']:.2f}")
    ax[0, i].axis('off')
    ax[1, i].imshow(row['ndvi'], cmap='coolwarm', vmin=-0.5, vmax=0.5)
    ax[1, i].axis('off')
plt.show()

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable
# plot a couple of random erf's
for row in df.sample(8).iterrows():
    plot_name = row[0]
    zone_im = row[1].clipped_cir
    zone_im_rgb = row[1].clipped_rgb
    print(plot_name)

    fig, axes = plt.subplots(1, 2, figsize=(12, 6))
    ndvi = calculate_ndvi(zone_im)

    divider = make_axes_locatable(axes[0])
    cax = divider.append_axes("right", size="5%", pad=0.05)
    
    im1 = axes[0].imshow(ndvi, cmap='coolwarm', vmin=-0.5, vmax=0.5)
    axes[0].set_title('NDVI')
    axes[0].axis('off')
    fig.colorbar(im1, cax=cax, orientation='vertical')

    axes[1].imshow(np.transpose(zone_im_rgb[:3], axes=(1, 2, 0)))
    axes[1].set_title('Normal Image')
    axes[1].axis('off')

    axes[0].set_aspect('equal')
    axes[1].set_aspect('equal')

    plt.tight_layout()
    plt.show()

In [ ]:
from rasterio.plot import show
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize

norm = Normalize(vmin=-0.5, vmax=0.5)  # Set the same vmin and vmax as in the plot
cmap = 'coolwarm'  # Same colormap as in the plot

fig, ax = plt.subplots(figsize=(12, 12))

for row in df.iterrows():
    plot_name = row[0]
    clipped_cir = row[1].clipped_cir
    clipped_rgb = row[1].clipped_rgb
    transform = row[1].affine_transform
    ndvi = row[1].ndvi
    divider = make_axes_locatable(axes[0])
    cax = divider.append_axes("right", size="5%", pad=0.05)

    # Plot the CIR raster
    show(
        ndvi,
        transform=transform,
        ax=ax,
        title="Combined Plot of Clipped Zones",
        cmap=cmap,  # Use a color map
        vmin=-0.5,
        vmax=0.5,
    )

# Set labels and titles
plt.xlabel("Longitude")
sm = ScalarMappable(cmap=cmap, norm=norm)
cbar = fig.colorbar(sm, ax=ax, orientation='vertical', fraction=0.02, pad=0.04)
plt.xlim(bounds[0], bounds[2])
plt.ylim(bounds[1], bounds[3])
plt.ylabel("Latitude")
plt.title("NDVI")
plt.axis("off")
plt.tight_layout()
plt.show()

In [ ]:
from rasterio.plot import show
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize

norm = Normalize(vmin=-0.5, vmax=0.5)  # Set the same vmin and vmax as in the plot
cmap = 'coolwarm'  # Same colormap as in the plot

fig, ax = plt.subplots(figsize=(12, 12))

df1 = results[1]
df2 = results[2]

for row in df1.iterrows():
    plot_name = row[0]
    clipped_cir = row[1].clipped_cir
    clipped_rgb = row[1].clipped_rgb
    transform = row[1].affine_transform
    ndvi1 = row[1].ndvi
    ndvi2 = df2.loc[plot_name]['ndvi']
    ndvi = ndvi2 - ndvi1
    divider = make_axes_locatable(axes[0])
    cax = divider.append_axes("right", size="5%", pad=0.05)

    # Plot the CIR raster
    show(
        ndvi,
        transform=transform,
        ax=ax,
        title="Combined Plot of Clipped Zones",
        cmap=cmap,  # Use a color map
        vmin=-0.5,
        vmax=0.5,
    )

# Set labels and titles
plt.xlabel("Longitude")
sm = ScalarMappable(cmap=cmap, norm=norm)
cbar = fig.colorbar(sm, ax=ax, orientation='vertical', fraction=0.02, pad=0.04)
plt.xlim(bounds[0], bounds[2])
plt.ylim(bounds[1], bounds[3])
plt.ylabel("Latitude")
plt.title("NDVI Change")
plt.axis("off")
plt.tight_layout()
plt.show()